In [3]:
from platform import python_version

print('python version of this jupyter notebook:', python_version())

python version of this jupyter notebook: 3.7.10


In [65]:
import pandas as pd
from tqdm.notebook import tqdm
from collections import defaultdict
import numpy as np

In [7]:
movies = pd.read_csv('./archive/IMDb movies.csv')

/Users/xuezhuoer/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
print('the length of the dataset is:', len(movies))

the length of the dataset is: 85855


In [9]:
# use 70000 of the dataset as the training set and the rest as the testing set
train_set = movies[:70000]
test_set = movies[70000:]

In [10]:
train_set.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [11]:
actor_movies =  {}
actor_count = {}
vote_add = 0
for index, row in tqdm(train_set.iterrows()):
    if type(row['actors']) != str:
        continue
    actors = row['actors'].split(',')
    for a in actors:
        if a not in actor_movies:
            actor_movies[a] = {row['imdb_title_id']}
            actor_count[a] = {0}
        else:
            actor_movies[a].add(row['imdb_title_id'])
            actor_count[a] 
    vote_add += row['avg_vote']
    
vote_mean = vote_add/len(train_set)

0it [00:00, ?it/s]

In [12]:
# count the number of movies an actor plays
actor_count = defaultdict(int)
actor_rate = defaultdict(int)

for index, row in tqdm(train_set.iterrows()):
    if type(row['actors']) != str:
        continue
    actors = row['actors'].split(',')
    for a in actors:
        a = a.replace(" ", "")
        actor_count[a] += 1
        actor_rate[a] += row['avg_vote']
        if a == "Angibell": print(row['title'], row['avg_vote'])

0it [00:00, ?it/s]

Lejos de Casa pelicula Venezolana 9.8


In [13]:
for a in actor_rate:
    actor_rate[a] = actor_rate[a]/actor_count[a]

In [14]:
actor_rate = sorted(actor_rate.items(), key= lambda x: x[1], reverse=True)

45


In [15]:
actor_count = sorted(actor_count.items(), key= lambda x: x[1], reverse=True)

In [59]:
# global count
# count = [0] * 10
def feata(d):
#     global count
    year = [0] * 11
    movie_year = d['year']
#     print(movie_year)
    if int(movie_year) <= 1930:
        year[0] = 1
#         count[0] += 1
    else:
        one_hot = int((int(movie_year) - 1921)/10)
        year[one_hot] = 1
#         count[one_hot] += 1
        
    return year[:-1]

In [60]:
X = [feata(row) for index, row in tqdm(train_set.iterrows())]
print(count)


[1057, 2840, 3014, 4144, 5151, 6192, 7432, 10159, 18684, 11327]


In [16]:
genres = []
genreSet = set()
for index, row in train_set.iterrows():
    genres.append([])
    for t in row['genre'].split(', '):
        genreSet.add(t)
        genres[-1].append(t)
numGenre = len(genreSet)
genreToIndex = dict()
for i, g in enumerate(list(genreSet)):
    genreToIndex[g] = i
for i, genre in enumerate(genres):
    t = [0] * numGenre
    for g in genre:
        t[genreToIndex[g]] = 1
    genres[i] = t
    
print(genres)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [62]:
countries = []
tmp = set()
frequency = defaultdict(int)
for index, row in train_set.iterrows():
    countries.append([])
#     print(index, row['country'], row['title'])
    if type(row['country']) != str: continue
    for t in row['country'].split(', '):
        tmp.add(t)
        frequency[t] += 1
        countries[-1].append(t)
numCountries = len(tmp)
frequency = sorted(frequency.keys(), key=lambda x: frequency[x], reverse=True)[:10]
print(frequency)
countryToIndex = dict()
for i, c in enumerate(frequency):
    countryToIndex[c] = i
for i, country in enumerate(countries):
    t = [0] * 10
    for c in country:
        if c in frequency:
            t[countryToIndex[c]] = 1
    countries[i] = t
    
# print(countries)

['USA', 'France', 'UK', 'Italy', 'India', 'Japan', 'Germany', 'Canada', 'Spain', 'Hong Kong']


In [96]:
print(countries[2000])
for index, row in train_set.iterrows():
    if index == 2000: 
        print(row['country'])
        break

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
USA


In [33]:
directorAndHisGenre = defaultdict(list)
m = 0

for index, row in train_set.iterrows():
    if type(row['director']) != str: continue 
    for t in row['director'].split(', '):
        if len(directorAndHisGenre[t]) == 0: directorAndHisGenre[t] = [0 for _ in range(2 * numGenre)]
        for g in row['genre'].split(', '):
            directorAndHisGenre[t][genreToIndex[g]] += 0.1
            directorAndHisGenre[t][genreToIndex[g]+numGenre] += row['avg_vote'] / 20
            m = max(m, directorAndHisGenre[t][genreToIndex[g]+numGenre])
        
print(directorAndHisGenre)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



print(m)
directorAndHisGenre['Paul Wegener']

In [27]:
genreStat = [sum([n[i] for n in genres]) / len(genres[0]) for i in range(len(genres[0]))]
print(genreStat)
genreToIndex
print(sum(genreStat))

[423.6, 126.28, 0.08, 1544.84, 481.08, 0.08, 0.04, 76.56, 34.48, 293.48, 166.88, 81.32, 383.48, 259.24, 58.6, 119.24, 75.16, 966.12, 26.52, 0.04, 63.76, 74.68, 346.6, 60.52, 126.76]
5789.440000000002


In [42]:
actorAndHisGenre = defaultdict(list)
m = 0

for index, row in train_set.iterrows():
    if type(row['actors']) != str: continue 
    for t in row['actors'].split(', '):
        tmpName = t.replace(' ', '')
        if len(actorAndHisGenre[tmpName]) == 0: actorAndHisGenre[tmpName] = [0 for _ in range(2 * numGenre)]
        for g in row['genre'].split(', '):
            actorAndHisGenre[tmpName][genreToIndex[g]] += 0.1
            actorAndHisGenre[tmpName][genreToIndex[g]+numGenre] += row['avg_vote'] / 20
#             m = max(m, directorAndHisGenre[t][genreToIndex[g]+numGenre])
        
print(actorAndHisGenre.keys())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [60]:
print(list(actorAndHisGenre.keys())[:10])

['BlancheBayliss', 'WilliamCourtenay', 'ChaunceyDepew', 'ElizabethTait', 'JohnTait', 'NormanCampbell', 'BellaCola', 'WillCoyne', 'SamCrewes', 'JackEnnis']


In [46]:
years = []
for index, row in tqdm(train_set.iterrows()):
    t = [0] * 10
    year = row['year']
    i = 9
    if int(year) <= 1930:
        i = 1
    else: 
        i = int((int(year) - 1921)/10)
    t[i] = 1
    years.append(t)

0it [00:00, ?it/s]

In [87]:
X = []
for index, row in tqdm(train_set.iterrows()):
    directorFeature = directorAndHisGenre[row['director']]
#     print(len(years[index]), len(directorFeature), len(genres[index]), len(countries[index]))
    X.append(years[index] + (directorFeature if len(directorFeature) > 0 else [0] * (2 * numGenre)) + genres[index] + countries[index])
    actorFeature = [0 for _ in range(2 * numGenre)]
    if type(row['actors']) == str: 
        for actor in row['actors'].split(', '):
            for i, x in enumerate(actorAndHisGenre[actor.replace(' ', '')]):
                actorFeature[i] += x
    X[-1] = X[-1] + actorFeature



0it [00:00, ?it/s]

In [64]:
y = []
for index, row in tqdm(train_set.iterrows()):
    y.append(row['avg_vote'])

0it [00:00, ?it/s]

In [97]:
X = np.array(X)
y = np.array(y)

In [106]:
import torch
from torch.nn import Module
from torch import nn, Tensor
import torch.functional as F

class RatingPredictor(Module):
    def __init__(self, input_dim):
        super(RatingPredictor, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x = self.__convert(x)
        o = self.layers(x)
        return o
        
    def __convert(self, x):
        if isinstance(x, Tensor): return x
        else: return Tensor(x)

In [107]:
EPOCHS = 1000
BATCH_SIZE = 5000

model = RatingPredictor(len(X[0]))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
MSE = nn.MSELoss()

trainSize = len(y)
for epoch in tqdm(range(EPOCHS)):
    randomIndex = np.random.choice(trainSize, size=BATCH_SIZE, replace=False)
#     print(X)
    pred = model(X[randomIndex])
    loss = MSE(pred, Tensor(y[randomIndex]))
    print(loss.detach().numpy().tolist())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  0%|          | 0/1000 [00:00<?, ?it/s]

39.07181930541992
139.8004913330078
25.65991973876953
34.266658782958984
35.316314697265625
34.39994430541992
34.162410736083984
31.96698570251465
26.82113265991211
18.48113441467285
16.86528205871582
15.805517196655273
13.308801651000977
13.10981273651123
13.350438117980957
12.402485847473145
11.399025917053223
11.56629467010498
10.537458419799805
9.660462379455566
9.268420219421387
8.6901216506958
8.287864685058594
7.366103172302246
7.083676338195801
6.196359157562256
5.932064533233643
5.3321123123168945
5.016836643218994
4.585527420043945
4.2825798988342285
3.930154800415039
3.6192142963409424
3.5022408962249756
3.3373303413391113
3.2656102180480957
3.2368738651275635
3.0651490688323975
2.872065544128418
2.88565731048584
2.777819871902466
2.6955456733703613
2.5939784049987793
2.528216600418091
2.4913649559020996
2.4221291542053223
2.417879104614258
2.3068766593933105
2.359020948410034
2.3147895336151123
2.359290599822998
2.332141876220703
2.152384042739868
2.342172145843506
2.535809

1.4862009286880493
1.547399878501892
1.508594274520874
1.539233922958374
1.4674150943756104
1.4830511808395386
1.4798766374588013
1.4841041564941406
1.544502854347229
1.4974608421325684
1.4737884998321533
1.470735788345337
1.4818357229232788
1.4371063709259033
1.517871379852295
1.494663953781128
1.442803144454956
1.5383814573287964
1.5442707538604736
1.4885448217391968
1.5345776081085205
1.553251028060913
1.4194070100784302
1.584397554397583
1.5364817380905151
1.522658109664917
1.4854971170425415
1.4840281009674072
1.4899195432662964
1.498473882675171
1.4258195161819458
1.5113072395324707
1.4410183429718018
1.5013206005096436
1.4904820919036865
1.4374958276748657
1.4961951971054077
1.462626576423645
1.4891289472579956
1.5199065208435059
1.515149474143982
1.5421011447906494
1.4813528060913086
1.4224588871002197
1.5502614974975586
1.5645568370819092
1.5161852836608887
1.5283805131912231
1.5413461923599243
1.4381499290466309
1.5949331521987915
1.570910096168518
1.4711329936981201
1.569747

1.450494408607483
1.495579481124878
1.5867573022842407
1.4216945171356201
1.530045747756958
1.5376839637756348
1.4782581329345703
1.5823513269424438
1.5321656465530396
1.413022756576538
1.507680058479309
1.5050594806671143
1.4892407655715942
1.4831302165985107
1.488978385925293
1.4499233961105347
1.4998393058776855
1.4961398839950562
1.416744589805603
1.429086446762085
1.464716911315918
1.4809778928756714
1.4471849203109741
1.4830914735794067
1.4552894830703735
1.4951499700546265
1.4894198179244995
1.4799808263778687
1.4450664520263672
1.4489854574203491
1.4914467334747314
1.4993878602981567
1.5178762674331665
1.5096652507781982
1.452899694442749
1.5871399641036987
1.4616508483886719
1.5164761543273926
1.4884448051452637
1.496383786201477
1.4513362646102905
1.5085527896881104
1.5418192148208618
1.4789035320281982
1.5114262104034424
1.5447463989257812
1.4556235074996948
1.4689217805862427
1.5096170902252197
1.5473570823669434
1.5115774869918823
1.4478825330734253
1.4823148250579834
1.53

In [88]:
for index, row in test_set.iterrows():
    p = model()

145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145


145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145


145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145


145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145


145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145


...

145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145


145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145


145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145


145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145


145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
145
